<a href="https://colab.research.google.com/github/amien1410/colab-notebooks/blob/main/Colab_Stock_Graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance plotly pandas

In [6]:
import yfinance as yf
import plotly.graph_objs as go
import pandas as pd

In [14]:
def fetch_data(tickers, start="2020-01-01", end=None):
    tickers = [t.strip().upper() for t in tickers]

    data = yf.download(tickers, start=start, end=end, group_by="ticker", auto_adjust=False)

    if len(tickers) == 1:
        # Single ticker returns flat dataframe
        ticker = tickers[0]
        if 'Adj Close' not in data.columns:
            raise KeyError(f"❌ 'Adj Close' not found for {ticker}.")
        df = data[['Adj Close']].rename(columns={'Adj Close': ticker})
    else:
        # Multi-ticker returns multi-level columns
        frames = []
        for ticker in tickers:
            try:
                adj_close = data[ticker]['Adj Close']
                frames.append(adj_close.rename(ticker))
            except Exception:
                print(f"⚠️ No data for {ticker}")
        if not frames:
            raise ValueError("❌ No valid stock data found.")
        df = pd.concat(frames, axis=1)

    return df

In [12]:
def plot_stocks(data, tickers):
    fig = go.Figure()

    for ticker in tickers:
        if ticker in data.columns:
            fig.add_trace(go.Scatter(
                x=data.index,
                y=data[ticker],
                mode='lines',
                name=ticker,
                line=dict(width=2)
            ))
        else:
            print(f"⚠️ Ticker not found in data: {ticker}")

    fig.update_layout(
        title="📈 Interactive Stock Price Chart",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        template="plotly_dark",
        height=600,
        xaxis=dict(
            rangeslider=dict(visible=True),
            rangeselector=dict(
                buttons=[
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ]
            )
        )
    )

    fig.show()

In [16]:
# ✅ Example usage:
tickers_input = input("Enter stock symbols separated by commas (e.g. AAPL,TSLA,NVDA): ")
tickers = [t.strip().upper() for t in tickers_input.split(",")]

try:
    df = fetch_data(tickers)
    plot_stocks(df, tickers)
except Exception as e:
    print(f"❌ Error: {e}")

Enter stock symbols separated by commas (e.g. AAPL,TSLA,NVDA): TSLA, NVDA, AAPL


[*********************100%***********************]  3 of 3 completed
